In [1]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     WRONG_VERSION_NUMBER] wrong version number
[nltk_data]     (_ssl.c:1129)>


False

In [194]:
import random
import spacy

from spacy.symbols import nsubj, VERB
from pattern.en import conjugate, PAST, PRESENT, tenses, parse, pprint, parsetree, SINGULAR, PLURAL
from itertools import tee

from datasets import load_dataset

nlp = spacy.load("en_core_web_lg")
dataset = load_dataset("samsum")

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     WRONG_VERSION_NUMBER] wrong version number
[nltk_data]     (_ssl.c:1129)>
[nltk_data] Error loading wordnet: HTTP Error 404: Not Found


ValueError: Error loading wordnet: HTTP Error 404: Not Found

In [12]:
def extract_speaker_names(data_list):
    speaker_names = set()
    for data in data_list:
        dialogue = data["dialogue"]
        for line in dialogue.split("\n"):
            speaker = line.split(":")[0].strip()
            if speaker:
                speaker_names.add(speaker)
    return list(speaker_names)

In [131]:
speaker_names = extract_speaker_names(dataset["train"])
pronouns = ["he", "she", "it", "they", "him", "her", "them", "his", "hers", "its", "theirs"]
tenses = ["past"]

In [127]:
def update_labels(labels, label_type, start_idx, end_idx, new_token):
    new_labels = []
    token_length = len(new_token.split())
    for i in range(len(labels)):
        if i < start_idx:
            new_labels.append(labels[i])
        elif i == start_idx:
            for j in range(token_length):
                new_labels.append(label_type)
        elif i > start_idx and i < end_idx:
            continue
        else:
            new_labels.append(labels[i])

    return new_labels

In [149]:
def replace_named_person(summary, dialog, labels):
    dialog_doc = nlp(dialog)
    dialog_entities = [ent.text for ent in dialog_doc.ents if ent.label_ in ["PERSON"]]
    
    summary_doc = nlp(summary)
    summary_entities = [ent for ent in summary_doc.ents if ent.label_ in ["PERSON"]]
    
    if len(summary_entities) == 0:
        return summary
    
    for entity in reversed(summary_entities):
        if random.random() < 0.5:
            continue  # Skip replacing this entity with a new name
        
        if random.random() < 0.5:
            new_name = random.choice(speaker_names)
            while new_name == entity.text:
                new_name = random.choice(speaker_names)
        else:
            new_name = random.choice(dialog_entities)
            while new_name == entity.text:
                new_name = random.choice(dialog_entities)
        
        labels = update_labels(labels, "E", entity.start, entity.end, new_name)
        summary = summary[:entity.start_char] + new_name + summary[entity.end_char:]
    
    return summary, labels

In [154]:
def replace_pronoun(summary, labels):
    summary_doc = nlp(summary)
    for token in reversed(summary_doc):
        if token.pos_ == "PRON":
            pronoun = token
            if random.random() < 0.5:
                continue  # Skip replacing this pronoun
            new_pronoun = random.choice(pronouns)
            while new_pronoun == pronoun.text:
                new_pronoun = random.choice(pronouns)
            
            labels = update_labels(labels, "P", pronoun.i, pronoun.i + 1, new_pronoun)
            summary = summary[:pronoun.idx] + new_pronoun + summary[pronoun.idx + len(pronoun):]
    
    return summary, labels

In [189]:
def replace_verb(summary, dialog, labels):
    # Extract verbs from the dialog
    dialog_doc = nlp(dialog)
    dialog_verbs = [token.lemma_ for token in dialog_doc if token.pos_ == 'VERB']

    # Replace verbs in the summary
    summary_doc = nlp(summary)
    for token in reversed(summary_doc):
        if token.pos_ == 'VERB':
            verb = token
            new_verb = random.choice(dialog_verbs)
            while new_verb == verb.text:
                new_verb = random.choice(dialog_verbs)
            
            labels = update_labels(labels, "V", verb.i, verb.i + 1, new_verb)
            summary = summary[:verb.idx] + new_verb + summary[verb.idx + len(verb):]

    return summary, labels

In [ ]:
def change_tense_spaCy(summary, to_tense):
    
    doc = nlp(unicode(text))
    
    out = []
    out.append(doc[0].text)
    for word_pair in pairwise(doc):
        if (word_pair[0].string == 'will' and word_pair[1].pos_ == u'VERB') \
        or word_pair[1].tag_ == u'VBD' or word_pair[1].tag_ == u'VBP':
            if to_tense == 'present':
                out.append(conjugate(word_pair[1].text, PRESENT))
            elif to_tense == 'past':
                out.append(conjugate(word_pair[1].text, PAST))
            elif to_tense == 'future':
                out.append('will')
                out.append(conjugate(word_pair[1].text, 'inf'))

            elif word_pair[1].text == 'will' and word_pair[1].tag_ == 'MD':
                pass
        else:
            out.append(word_pair[1].text)


    text_out = ' '.join(out)

    for char in string.punctuation:
        if char in """(<['‘""":
            text_out = text_out.replace(char+' ',char)
        else:
            text_out = text_out.replace(' '+char,char)

    text_out = text_out.replace(" 's","'s") #fix posessive 's
    
    return text_out

In [190]:
data = dataset["train"][0]
summary_doc = nlp("Michael Oliver baked cookies and will bring Jerry some tomorrow . He is really happy now")
summary = " ".join([token.text for token in summary_doc])
labels = ["O"] * len(summary.split())
distorted, labels = replace_named_person(summary, data["dialogue"], labels)
print(labels)
distorted, labels = replace_pronoun(distorted, labels)
distorted, labels = replace_verb(distorted, data["dialogue"], labels)
# distorted = "Michael Oliver baked cookies and will bring Jerry some tomorrow."
print(distorted)
print(len(distorted.split()))
print(labels)
print(len(labels))
# print(distorted[14:])

['E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Amelia bake cookies and will bake Jerry some tomorrow . He is really happy now
15
['E', 'V', 'O', 'O', 'O', 'V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
15


In [180]:
doc = nlp("Michael Oliver give back cookies and will bring Jerry some tomorrow . He is really happy now")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Michael Michael PROPN NNP compound Xxxxx True False
Oliver Oliver PROPN NNP nsubj Xxxxx True False
give give VERB VB ROOT xxxx True True
back back ADP RP prt xxxx True True
cookies cookie NOUN NNS dobj xxxx True False
and and CCONJ CC cc xxx True True
will will AUX MD aux xxxx True True
bring bring VERB VB conj xxxx True False
Jerry Jerry PROPN NNP dative Xxxxx True False
some some DET DT det xxxx True True
tomorrow tomorrow NOUN NN npadvmod xxxx True False
. . PUNCT . punct . False False
He he PRON PRP nsubj Xx True True
is be AUX VBZ ROOT xx True True
really really ADV RB advmod xxxx True True
happy happy ADJ JJ acomp xxxx True False
now now ADV RB advmod xxx True True


In [139]:
len("AutoModelForSeq2SeqLM")

21